In [12]:
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch
import torch.nn as nn
from torchvision import models
import torch.optim as optim

In [13]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


In [14]:
data = pd.read_csv("data_normalized.csv")

# Create a mapping for categories
unique_categories = data['Category'].unique()
category_mapping = {category: idx for idx, category in enumerate(unique_categories)}
category_mapping


{'Unripe': 0, 'overripe': 1, 'ripe': 2}

In [15]:
from sklearn.preprocessing import MinMaxScaler

# Load CSV
data = pd.read_csv('topg.csv')

# Normalize specific columns
columns_to_normalize = ['Freshness', 'Shape and Size', 'Texture', 'Color', 'Firmness', 
                        'Batch Consistency', 'Overall Quality']
scaler = MinMaxScaler()
data[columns_to_normalize] = scaler.fit_transform(data[columns_to_normalize])

# Save normalized data if needed
data.to_csv('data_normalized.csv', index=False)
print("Normalized CSV saved.")


Normalized CSV saved.


In [16]:
class ImageDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.image_dir = image_dir
        self.transform = transform

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        image_path = os.path.join(self.image_dir, row['file_path'])
        image = Image.open(image_path).convert("RGB")
        category_idx = category_mapping[row['Category']]
        if self.transform:
            image = self.transform(image)

        labels = {
            'Category': torch.tensor(category_idx, dtype=torch.long),  # Long tensor for category            'Freshness': torch.tensor(row['Freshness'], dtype=torch.float),  # Float tensor for regression
            'Shape_and_Size': torch.tensor(row['Shape and Size'], dtype=torch.float),
            'Texture': torch.tensor(row['Texture'], dtype=torch.float),
            'Color': torch.tensor(row['Color'], dtype=torch.float),
            'Firmness': torch.tensor(row['Firmness'], dtype=torch.float),
            'Batch_Consistency': torch.tensor(row['Batch Consistency'], dtype=torch.float),
            'Damaged': torch.tensor(row['Damaged'], dtype=torch.float),  # Float tensor for binary classification
            'Overall_Quality': torch.tensor(row['Overall Quality'], dtype=torch.float),
        }

        return image, labels


    def __len__(self):
        return len(self.data)


In [17]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)

In [18]:
# Load dataset  
image_dir = ''
csv_file = 'data_normalized.csv'
dataset = ImageDataset(csv_file=csv_file, image_dir=image_dir, transform=transform)

# Split into training and validation sets
from torch.utils.data import random_split

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=10, shuffle=False)


In [19]:
# Add error handling and logging
try:
    for images, labels in train_loader:
        images = images.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
        
        # Debugging: Print shapes
        print("Image shape:", images.shape)
        for key, value in labels.items():
            print(f"{key} shape: {value.shape}")
        
        # Rest of the training code...
except Exception as e:
    print(f"Error during training: {e}")

Image shape: torch.Size([10, 3, 224, 224])
Category shape: torch.Size([10])
Shape_and_Size shape: torch.Size([10])
Texture shape: torch.Size([10])
Color shape: torch.Size([10])
Firmness shape: torch.Size([10])
Batch_Consistency shape: torch.Size([10])
Damaged shape: torch.Size([10])
Overall_Quality shape: torch.Size([10])
Image shape: torch.Size([10, 3, 224, 224])
Category shape: torch.Size([10])
Shape_and_Size shape: torch.Size([10])
Texture shape: torch.Size([10])
Color shape: torch.Size([10])
Firmness shape: torch.Size([10])
Batch_Consistency shape: torch.Size([10])
Damaged shape: torch.Size([10])
Overall_Quality shape: torch.Size([10])
Image shape: torch.Size([10, 3, 224, 224])
Category shape: torch.Size([10])
Shape_and_Size shape: torch.Size([10])
Texture shape: torch.Size([10])
Color shape: torch.Size([10])
Firmness shape: torch.Size([10])
Batch_Consistency shape: torch.Size([10])
Damaged shape: torch.Size([10])
Overall_Quality shape: torch.Size([10])
Image shape: torch.Size([10,

In [20]:
# Check pixel value statistics
for i in range(3):  # Check first 3 samples
    image, label = train_dataset[i]
    print(f"Image Mean: {image.mean():.4f}, Image Std: {image.std():.4f}, Label: {label}")


Image Mean: 0.5182, Image Std: 0.9516, Label: {'Category': tensor(2), 'Shape_and_Size': tensor(0.8000), 'Texture': tensor(1.), 'Color': tensor(0.8333), 'Firmness': tensor(0.8889), 'Batch_Consistency': tensor(0.8571), 'Damaged': tensor(0.), 'Overall_Quality': tensor(0.8780)}
Image Mean: 2.2041, Image Std: 0.7867, Label: {'Category': tensor(0), 'Shape_and_Size': tensor(0.4000), 'Texture': tensor(0.7143), 'Color': tensor(0.6667), 'Firmness': tensor(0.6667), 'Batch_Consistency': tensor(0.7143), 'Damaged': tensor(0.), 'Overall_Quality': tensor(0.6098)}
Image Mean: 2.1634, Image Std: 0.6671, Label: {'Category': tensor(0), 'Shape_and_Size': tensor(0.4000), 'Texture': tensor(0.7143), 'Color': tensor(0.6667), 'Firmness': tensor(0.7778), 'Batch_Consistency': tensor(0.5714), 'Damaged': tensor(0.), 'Overall_Quality': tensor(0.6098)}


In [21]:
# Model Definition
class MultiOutputResNet(nn.Module):
    def __init__(self, num_categories):
        super(MultiOutputResNet, self).__init__()
        self.base_model = models.resnet50(pretrained=True)
        num_features = self.base_model.fc.in_features
        
        # Replace the classification head with task-specific heads
        self.base_model.fc = nn.Identity()  # Remove original classification layer
        self.category_head = nn.Linear(num_features, num_categories)  # For Category
        self.shape_size_head = nn.Linear(num_features, 1)  # Regression
        self.texture_head = nn.Linear(num_features, 1)  # Regression
        self.color_head = nn.Linear(num_features, 1)  # Regression
        self.firmness_head = nn.Linear(num_features, 1)  # Regression
        self.batch_consistency_head = nn.Linear(num_features, 1)  # Regression
        self.damaged_head = nn.Linear(num_features, 1)  # Binary classification
        self.quality_head = nn.Linear(num_features, 1)  # Regression

    def forward(self, x):
        features = self.base_model(x)
        
        return {
            'Category': self.category_head(features),
            'Shape_and_Size': self.shape_size_head(features),
            'Texture': self.texture_head(features),
            'Color': self.color_head(features),
            'Firmness': self.firmness_head(features),
            'Batch_Consistency': self.batch_consistency_head(features),
            'Damaged': torch.sigmoid(self.damaged_head(features)),  # Sigmoid for binary output
            'Overall_Quality': self.quality_head(features)
        }


In [22]:
num_categories = len(dataset.data['Category'].unique())
model = MultiOutputResNet(num_categories)
model = model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
num_categories


/home/afnaan/Desktop/labeling/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/afnaan/Desktop/labeling/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


3

In [23]:
criterion_category = nn.CrossEntropyLoss()
criterion_regression = nn.MSELoss()
criterion_binary = nn.BCEWithLogitsLoss()
# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)


In [24]:
# Training Loop
num_epochs = 10
accumulation_steps = 4  # Number of steps to accumulate gradients
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (images, labels) in enumerate(val_loader):
        images = images.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
        
        # Forward pass
        outputs = model(images)
        
        # Debugging: Print shapes and values
        print(f"Epoch {epoch+1}, Batch {i+1}")
        print(f"outputs['Category'] shape: {outputs['Category'].shape}, labels['Category'] shape: {labels['Category'].shape}")
        print(f"outputs['Category'] values: {outputs['Category']}")
        print(f"labels['Category'] values: {labels['Category']}")
        
        # Compute Loss
        loss_category = criterion_category(outputs['Category'], labels['Category'].long().to(images.device))
        loss_shape_size = criterion_regression(outputs['Shape_and_Size'], labels['Shape_and_Size'].float().to(images.device))
        loss_texture = criterion_regression(outputs['Texture'], labels['Texture'].float().to(images.device))
        loss_color = criterion_regression(outputs['Color'], labels['Color'].float().to(images.device))
        loss_firmness = criterion_regression(outputs['Firmness'], labels['Firmness'].float().to(images.device))
        loss_batch_consistency = criterion_regression(outputs['Batch_Consistency'], labels['Batch_Consistency'].float().to(images.device))
        loss_damaged = criterion_binary(outputs['Damaged'], labels['Damaged'].float().unsqueeze(1).to(images.device))
        loss_quality = criterion_regression(
            outputs['Overall_Quality'], 
            labels['Overall_Quality'].float().view(-1, 1).to(images.device)
        )

        total_loss = (
            loss_category + loss_shape_size + loss_texture + 
            loss_color + loss_firmness + loss_batch_consistency + 
            loss_damaged + loss_quality
        )

        # Backward pass and optimization
        total_loss = total_loss / accumulation_steps
        total_loss.backward()
        
        if (i + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
        
        running_loss += total_loss.item()
        
        # Clear cache to avoid OOM
        torch.cuda.empty_cache()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(val_loader):.4f}")


Epoch 1, Batch 1
outputs['Category'] shape: torch.Size([10, 3]), labels['Category'] shape: torch.Size([10])
outputs['Category'] values: tensor([[ 0.2372, -0.6451,  0.3219],
        [-0.1595, -0.7402, -0.0879],
        [ 0.0734, -0.3081,  0.1005],
        [-0.4236, -0.6393, -0.0669],
        [ 0.0250, -0.4056,  0.2265],
        [ 0.1022, -0.4359, -0.3173],
        [ 0.4041, -0.5590, -0.1423],
        [-0.1318, -0.2892, -0.1394],
        [ 0.3834, -0.3910, -0.0580],
        [ 0.2472, -0.8049, -0.3480]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
labels['Category'] values: tensor([0, 1, 2, 1, 0, 2, 0, 1, 0, 0])


/home/afnaan/Desktop/labeling/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1, Batch 2
outputs['Category'] shape: torch.Size([10, 3]), labels['Category'] shape: torch.Size([10])
outputs['Category'] values: tensor([[ 0.0659, -0.2939,  0.0423],
        [ 0.1435, -0.4644, -0.0700],
        [ 0.2281, -0.6605, -0.0263],
        [ 0.2630, -0.6581,  0.0441],
        [ 0.0946, -0.3916, -0.1993],
        [-0.2560, -0.4946, -0.0995],
        [-0.1219, -0.5443, -0.0906],
        [-0.0434, -0.5165, -0.0688],
        [ 0.2612, -0.5274,  0.1661],
        [ 0.0349, -0.5724, -0.2702]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
labels['Category'] values: tensor([1, 0, 0, 0, 2, 1, 0, 1, 1, 1])
Epoch 1, Batch 3
outputs['Category'] shape: torch.Size([10, 3]), labels['Category'] shape: torch.Size([10])
outputs['Category'] values: tensor([[-0.1227, -0.4819,  0.1105],
        [-0.0181, -0.9220, -0.3242],
        [ 0.0677, -0.1843, -0.1131],
        [ 0.1161, -0.4336,  0.2400],
        [ 0.3638, -0.3128, -0.0838],
        [ 0.2715, -0.4595,  0.0661],
        [-0.4618, -

/home/afnaan/Desktop/labeling/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 2, Batch 1
outputs['Category'] shape: torch.Size([10, 3]), labels['Category'] shape: torch.Size([10])
outputs['Category'] values: tensor([[-0.2497, -0.7356,  0.5572],
        [-0.6178, -0.8622,  0.9600],
        [-0.8781, -0.9088,  1.1014],
        [-1.9157,  0.9282,  0.3676],
        [ 0.4040, -0.8659,  0.0664],
        [-0.9814, -1.1690,  1.2015],
        [ 0.8706, -0.8010, -0.5480],
        [-0.1687, -1.0621,  0.2812],
        [ 0.4956, -0.6906, -0.2337],
        [ 0.3640, -1.4749, -0.7945]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
labels['Category'] values: tensor([0, 1, 2, 1, 0, 2, 0, 1, 0, 0])
Epoch 2, Batch 2
outputs['Category'] shape: torch.Size([10, 3]), labels['Category'] shape: torch.Size([10])
outputs['Category'] values: tensor([[-0.8745, -0.6362,  1.1143],
        [ 0.9360, -1.6787, -0.8676],
        [-0.4588, -0.6679,  0.4759],
        [ 0.5269, -1.3226,  0.3226],
        [-0.6537, -1.0278,  1.2179],
        [-1.2548,  0.1830,  0.3570],
        [-0.6509, -

In [25]:
def validate_model(model, val_dataloader, device):
    model.eval()  # Set model to evaluation mode
    total_loss = 0
    num_samples = 0
    
    criterion_category = nn.CrossEntropyLoss()
    criterion_regression = nn.MSELoss()
    criterion_binary = nn.BCELoss()

    with torch.no_grad():  # No gradient calculation
        for images, labels in val_dataloader:
            images = images.to(device)
            
            # Forward pass
            outputs = model(images)
            
            loss_category = criterion_category(outputs['Category'], labels['Category'].long().to(images.device))
            loss_shape_size = criterion_regression(outputs['Shape_and_Size'], labels['Shape_and_Size'].float().to(images.device))
            loss_texture = criterion_regression(outputs['Texture'], labels['Texture'].float().to(images.device))
            loss_color = criterion_regression(outputs['Color'], labels['Color'].float().to(images.device))
            loss_firmness = criterion_regression(outputs['Firmness'], labels['Firmness'].float().to(images.device))
            loss_batch_consistency = criterion_regression(outputs['Batch_Consistency'], labels['Batch_Consistency'].float().to(images.device))
            loss_damaged = criterion_binary(outputs['Damaged'], labels['Damaged'].float().unsqueeze(1).to(images.device))
            loss_quality = criterion_regression(
                outputs['Overall_Quality'], 
                labels['Overall_Quality'].float().view(-1, 1).to(images.device)
        )            
            # Total loss
            total_batch_loss = (
                loss_category + loss_shape_size + loss_texture + 
                loss_color + loss_firmness + loss_batch_consistency + 
                loss_damaged + loss_quality
            )
            print(f"Validation Loss: {total_batch_loss.item():.4f}")
            total_loss += total_batch_loss.item() * images.size(0)  # Accumulate loss
            num_samples += images.size(0)
    
    avg_loss = total_loss / num_samples
    print(f"Loss: {running_loss / len(val_loader)}")


In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
validate_model(model, val_loader, device)

Validation Loss: 1.2798
Validation Loss: 3.2403
Validation Loss: 1.1765
Validation Loss: 0.9682
Validation Loss: 2.4783
Validation Loss: 1.2210
Validation Loss: 2.4224
Validation Loss: 1.9946
Validation Loss: 1.4113
Validation Loss: 2.9532
Validation Loss: 2.5562
Validation Loss: 2.3952
Validation Loss: 2.1567
Validation Loss: 1.3153
Validation Loss: 0.2792
Validation Loss: 0.3763
Validation Loss: 1.2500
Validation Loss: 1.0449
Validation Loss: 0.2646
Validation Loss: 0.2716
Loss: 0.2675023630261421


In [27]:
import torch
from torchvision import transforms
from PIL import Image, ImageFile

# Allow loading of truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Path to your random image
random_image_path = "0629.jpg"

# Preprocessing (same as used during training/validation)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Ensure the image size matches the input size of the model
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Same normalization as training
])

# Load and preprocess the image
image = Image.open(random_image_path).convert("RGB")
input_image = transform(image).unsqueeze(0)  # Add batch dimension

# Move image to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_image = input_image.to(device)

# Ensure model is in evaluation mode
model.eval()

# Inference
with torch.no_grad():
    output = model(input_image)
category = torch.argmax(output["Category"], dim=1).item()
category_name = list(category_mapping.keys())[list(category_mapping.values()).index(category)]

# Interpret the output
output_dict = {
    "Category": category_name,  # Multi-class classification
    "Shape_and_Size": output["Shape_and_Size"].item(),
    "Texture": output["Texture"].item(),
    "Color": output["Color"].item(),
    "Firmness": output["Firmness"].item(),
    "Batch_Consistency": output["Batch_Consistency"].item(),
    "Damaged": torch.round(output["Damaged"]).item(),  # Binary classification (round to 0 or 1)
    "Overall_Quality": output["Overall_Quality"].item()
}


# Print the results
print("Predicted Parameters:")
for key, value in output_dict.items():
    print(f"{key}: {value}")


Predicted Parameters:
Category: Unripe
Shape_and_Size: 0.636222779750824
Texture: 0.5969353318214417
Color: 0.5918222069740295
Firmness: 0.6659048795700073
Batch_Consistency: 0.6495285034179688
Damaged: 0.0
Overall_Quality: 0.7727771401405334


In [28]:
torch.save(model.state_dict(), 'complete_model.pt')